# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

 
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob( os.path.join(root,'*') )
    
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:   
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
   

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This will make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
#Creating a Keyspace 
try:
    query = "Create Keyspace if Not exists udacity with replication = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"
    session.execute(query)
except Exception as e:
    print(e)


#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
##Creating table for query no 1
##Since the data needs to be structured and ordered based on the sessionId and itemInSession, those are the partition_key and clustering columns. 

try:
    query = "CREATE TABLE IF NOT EXISTS Q1_art_song_len( sessionId int , itemInSession int, artist text, song text, length double , PRIMARY KEY (sessionId , itemInSession ))"
    session.execute(query)
except Exception as e:
    print(e)

                    

In [13]:

file = 'event_datafile_new.csv'

##code used to open the 'event_datafile_new.csv' file and extract data and then load it in the Q1_art_song_len table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  
    
    for line in csvreader:
         query = "INSERT INTO Q1_art_song_len ( sessionId , itemInSession, artist , song , length ) VALUES (%s, %s, %s, %s, %s) "
         session.execute( query, ( int(line[8]) , int(line[3]), line[0], line[9], float(line[5])  ) )
         
    

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## the select query 
query11 = "select artist, song , length from Q1_art_song_len where sessionId = 338 and itemInSession = 4"

try:
    rows1 = session.execute(query11) 
    
    for row in rows1:
        print(row)
        
except Exception as e:
    print(e)
    
 

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [10]:
##Creating table for query no 2
##The userId and sessionId does not account for a unique partition_key and clustering column, 
##hence the Table has column itemInSession as the clustering column as well for partitioning.
##itemInSession has been used in the query for sorting the song in DESC order further based on itemInSession

try:
    query = "CREATE TABLE IF NOT EXISTS Q2_art_song_user( \
             userId int, sessionId int,  itemInSession int, artist text, song text, firstName text, lastName text, \
             PRIMARY KEY ( ( userId, sessionId ) , itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC);"
    session.execute(query)
except Exception as e:
    print(e)
    

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
         query = "INSERT INTO Q2_art_song_user ( userId , sessionId ,  itemInSession , artist , song , firstName , lastName ) VALUES (%s, %s, %s, %s, %s, %s, %s) "
         session.execute( query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[9],  line[1], line[4]  ) )
          

            
query12 = "select artist , song , firstname , lastname from Q2_art_song_user \
           where userId = 10 and sessionId = 182"

try:
    rows1 = session.execute(query12) 
    
    for row in rows1:
        print(row)
        
except Exception as e:
    print(e)
    

                    

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


In [15]:
##Creating table for query no 3
##song is the main partitioning key over here. FirstName and lastName can serve as proper clustering columns cause they are unique.
##userId has been used as a clustering column , if at all there is a chance that 2 users might have same firstname and lastname.

try:
    query = "CREATE TABLE IF NOT EXISTS Q3_user_song( song text , userId int, firstName text, lastName text, \
             PRIMARY KEY (song , userId ) )"
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  
    
    for line in csvreader:
         query = "INSERT INTO Q3_user_song ( song ,userId ,firstName , lastName ) VALUES (%s, %s, %s, %s) "
         session.execute( query, ( line[9], int(line[10]) , line[1], line[4]  ) )
         

query13 = "select song, firstname , lastname from Q3_user_song where song = 'All Hands Against His Own' "


try:
    rows1 = session.execute(query13) 
    
    for row in rows1:
        print(row)
        
except Exception as e:
    print(e)           
 
    
       

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
##Dropping the 3 tables before closing out the sessions

In [16]:
query_1 = "drop table Q1_art_song_len"

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
   

   
query_2 = "drop table Q2_art_song_user"

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    

    
    
query_3 = "drop table Q3_user_song"

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()